#
<span style="font-family: Times New Roman; font-size: 20px;">
<h1 align='center'> 
Tensorboard Implementation
</h1>

##
<span style="font-family: Times New Roman; font-size: 20px;">
<h2> 
Imports
</h2>

In [19]:
import tensorflow as tf
import matplotlib.pyplot as plt
import io,os,shutil
import numpy as np
import torch
from torch import nn
import torchvision
from sklearn.metrics import accuracy_score, confusion_matrix
from torchvision.transforms import transforms
import warnings
warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##
<span style="font-family: Times New Roman; font-size: 20px;">
<h2> 
Model Architecture
</h2>

###
<span style="font-family: Times New Roman; font-size: 20px;">
<h3> 
VGG-Model (1 Block)
</h3>

In [20]:
class VGG1Block(nn.Module):
  def __init__(self):
    super(VGG1Block, self).__init__()
    self.features = nn.Sequential(
      nn.Conv2d(3, 32, 3, padding=1),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(2, 2)
    )
    self.classifier = nn.Sequential(
      nn.Linear(32 * 128 * 128, 256),
      nn.ReLU(inplace=True),
      nn.Linear(256, 16),
      nn.ReLU(inplace=True),
      nn.Linear(16, 1)
    )

  def forward(self, x):
    x = self.features(x)
    x = x.view(x.size(0), -1)
    x = self.classifier(x)
    return x

###
<span style="font-family: Times New Roman; font-size: 20px;">
<h3> 
VGG-Model (3 Blocks)
</h3>

In [21]:
class VGG3Block(nn.Module):
    def __init__(self, *args, **kwargs):
        super(VGG3Block, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 32 * 32, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 16),
            nn.ReLU(inplace=True),
            nn.Linear(16, 1)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

###
<span style="font-family: Times New Roman; font-size: 20px;">
<h3> 
VGG16-Model (Transfer Learning) (only MLP layers)
</h3>

In [22]:
from torchvision.models import vgg16

class VGG16_tl(nn.Module):
    def __init__(self):
        super(VGG16_tl, self).__init__()
        self.base_model = vgg16(pretrained=True)
        for param in self.base_model.parameters():
            param.requires_grad = False
        self.classifier = nn.Sequential(
            nn.Linear(1000, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 16),
            nn.ReLU(inplace=True),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        x = self.base_model(x)
        x = self.classifier(x)
        return x

###
<span style="font-family: Times New Roman; font-size: 20px;">
<h3> 
VGG16-Model (Transfer Learning) (All the layers)
</h3>

In [23]:
from torchvision.models import vgg16

class VGG16_tl_all(nn.Module):
    def __init__(self):
        super(VGG16_tl_all, self).__init__()
        self.base_model = vgg16(pretrained=True)
        self.classifier = nn.Sequential(
            nn.Linear(1000, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 16),
            nn.ReLU(inplace=True),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        x = self.base_model(x)
        x = self.classifier(x)
        return x

##
<span style="font-family: Times New Roman; font-size: 20px;">
<h2> 
Loading the Test Data Iterator
</h2>

In [24]:
logdir = "logs/"

In [25]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])
tester = torchvision.datasets.ImageFolder(root='data/test', transform=transform)
tester_it = torch.utils.data.DataLoader(tester, batch_size=1, shuffle=True)

##
<span style="font-family: Times New Roman; font-size: 20px;">
<h2> 
Function to Log Images to Tensorboard
</h2>

In [26]:
def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)
 
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image

In [27]:
def log_images(model, itr, writer, num_of_images, class_names=None, k=2):
    if class_names is None:
        class_names = [f"Class {i}" for i in range(k)]
    k = len(class_names)
    
    pred_labels = []
    true_labels = []
    num = num_of_images // 5
    
    figure = plt.figure(figsize=(12,2*num))   
    plt.suptitle("Predictions on Test Data")
    plt.margins(0.5)
    for i, (image, label) in enumerate(itr):
        true_labels.append(label.item())
        prediction = model(image).detach().numpy()
        pred_labels.append(int(prediction > 0))
        plt.subplot(num, 5, i + 1, title=f"{class_names[int(prediction > 0)]} ({'Right' if class_names[label.item()]==class_names[int(prediction > 0)] else 'Wrong'})")
        plt.xticks([])
        plt.yticks([])
        plt.grid(False) 
        plt.imshow((image.numpy()).transpose((0, 2, 3, 1)).squeeze(), cmap=plt.cm.binary)
    plt.tight_layout()
    with writer.as_default():
        tf.summary.image("Predictions on Test Data", plot_to_image(figure), step=0, description="Image")
    
    cm = confusion_matrix(true_labels, pred_labels)
    fig = plt.figure(figsize=(4, 4))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    for i in range(k):
        for j in range(k):
            plt.text(j, i, cm[i, j], horizontalalignment="center", color="black")
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    with writer.as_default():
        tf.summary.image("Confusion Matrix", plot_to_image(fig), step=0, description="Confusion Matrix")
    return pred_labels, true_labels


##
<span style="font-family: Times New Roman; font-size: 20px;">
<h2> 
Loading the Model
</h2>

In [28]:
model_path = ["model.pth", "model2.pth", "model3.pth", "model4.pth", "model5.pth"]
models = []
model = VGG1Block().to(device)
model.load_state_dict(torch.load(model_path[0], map_location=device))
model.eval()
models.append(model)
for i in range(1, 3):
    model = VGG3Block().to(device)
    model.load_state_dict(torch.load(model_path[i], map_location=device))
    model.eval()
    models.append(model)
model = VGG16_tl().to(device)
model.load_state_dict(torch.load(model_path[3], map_location=device).state_dict())
model.eval()
models.append(model)
model = VGG16_tl_all().to(device)
model.load_state_dict(torch.load(model_path[4], map_location=device).state_dict())
model.eval()
models.append(model)

In [29]:
len(models)

5

In [30]:
classes = ["Cheetah", "Jaguar"]

In [31]:
if os.path.exists(logdir):
    shutil.rmtree(logdir)
for i,model in enumerate(models):
    logdir = f"logs/model{i+1}/"
    writer = tf.summary.create_file_writer(logdir)
    pred_labels, true_labels = log_images(model, iter(tester_it), writer, len(tester_it), classes)
    print(f"For Model{i+1}")
    print(f"Accuracy: {accuracy_score(true_labels, pred_labels)*100}%")
    writer.close()

For Model1
Accuracy: 95.0%
For Model2
Accuracy: 90.0%
For Model3
Accuracy: 90.0%
For Model4
Accuracy: 97.5%
For Model5
Accuracy: 100.0%


In [32]:
%tensorboard

UsageError: Line magic function `%tensorboard` not found.
